In [1]:
import numpy as np
data=np.array([[2,2,1],[2,1,2],[2,0,0],[1,2,1],[1,1,1],[1,0,0],
               [1,2,2],[0,1,2],[0,1,0],[0,0,1],[0,0,2]])
label=np.array([1,0,1,1,1,1,1,0,1,0,0])
# 需要使用sklearn版本 0.22以上
from sklearn.naive_bayes import CategoricalNB
mnb = CategoricalNB()
mnb.fit(data,label)
test_sample =[[1,1,0]]
print("测试结果:", mnb.predict_proba(test_sample))


测试结果: [[0.0768935 0.9231065]]


In [2]:
"""
在bagging(随机森林)，
和boosting(AdaBoost)中用不同数目的学习器
（例如10,50,100）进行学习训练，打印结果的准确性
"""
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#随机森林
from sklearn.ensemble import RandomForestClassifier
nb_trees=[10,20,30,40,50,60,70,80,90,100]
for nb_tree in nb_trees:
    print('--------------------------------------')
    print(f'随机森林在决策树为{nb_tree}的情况下进行训练的结果')
    rf=RandomForestClassifier(n_jobs=-1,n_estimators=nb_tree)
    rf.fit(X_train, y_train)
    rf_acc=rf.score(X_test, y_test)
    print(f'准确率为{rf_acc}')

#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
nb_trees=[100,200,300,400,500,600,700,800]
for nb_tree in nb_trees:
    print('--------------------------------------')
    print(f'AdaBoost在决策树为{nb_tree}的情况下进行训练的结果')
    clf = AdaBoostClassifier(n_estimators=nb_tree, learning_rate=0.5)
    clf.fit(X_train, y_train)
    clf_acc=clf.score(X_test, y_test)
    print(f'准确率为{clf_acc}')

--------------------------------------
随机森林在决策树为10的情况下进行训练的结果
准确率为0.96
--------------------------------------
随机森林在决策树为20的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
随机森林在决策树为30的情况下进行训练的结果
准确率为0.9777777777777777
--------------------------------------
随机森林在决策树为40的情况下进行训练的结果
准确率为0.9711111111111111
--------------------------------------
随机森林在决策树为50的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
随机森林在决策树为60的情况下进行训练的结果
准确率为0.9733333333333334
--------------------------------------
随机森林在决策树为70的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
随机森林在决策树为80的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
随机森林在决策树为90的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
随机森林在决策树为100的情况下进行训练的结果
准确率为0.9755555555555555
--------------------------------------
AdaBoost在决策树为100的情况下进行训练的结果
准确率为0.6711111111111111
--------------------------------------
AdaBoost在决策树为200的情况下进行训练的结果
准确率为0.

In [3]:
"""
1实现后向特征选择法
"""
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import copy
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
mse_dic = mean_squared_error(y_train, y_pred)

num_feature = X_train.shape[1]
feature_li=list(range(0,num_feature))
print(f"所有特征{feature_li}都进行训练模型的MSE误差为：{mse_dic}")
last_mse = mse_dic
for i in range(len(feature_li)):#一共有feature_li_num个数量的变量等待剔除
    remove_mse_dic = {}
    for featu in feature_li:
        if len(feature_li) < 2: break
        feature_li_wait_remove = copy.deepcopy(feature_li)
        # 去掉features组合中一个特征
        feature_li_wait_remove.remove(featu)
        X_in = X_train[:, feature_li_wait_remove]
        reg.fit(X_in, y_train)
        y_pred = reg.predict(X_in)
        remove_mse_dic[featu] = mean_squared_error(y_train, y_pred)
        #print(f"用特征{feature_li_wait_remove}进行训练模型的MSE误差为：{remove_mse_dic[featu]}")
    if remove_mse_dic :
            max_fea = min(remove_mse_dic, key=remove_mse_dic.get)
    if remove_mse_dic[max_fea] < last_mse:
            last_mse = remove_mse_dic[max_fea]
            feature_li.remove(max_fea)
            print("第" + str(i+1 ) + "轮剔除特征",max_fea,"MSE：", last_mse)
            print("剔除特征后的features",feature_li)
            #else:
                #break
print("挑选后的特征集合：",feature_li)
print("挑选特征后的训练集MSE：",last_mse)

reg.fit(X_test, y_test)
y_pred = reg.predict(X_test)
mse_dic = mean_squared_error(y_test, y_pred)
print("后向选择测试集MSE：",mse_dic)

所有特征[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]都进行训练模型的MSE误差为：2907.253864307293
挑选后的特征集合： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
挑选特征后的训练集MSE： 2907.253864307293
后向选择测试集MSE： 2373.808326173681


In [4]:
"""
2在梯度下降法中修改SimpleLinearRegression中的代码，
使得在fit函数中能显示每一次迭代得到的参数的MSE误差是多少，并使用load_diabetes数据集。
"""
class SimpleLinearRegression:

    def __init__(self):
        self.theta = None

    def fit(self, X_train, y_train, learning_rate=0.02, n_iters=1e5, epsilon=1e-8):
        #计算目标函数
        def J(theta):
            return 1/2.0*np.sum((np.dot(X_train, theta) - y_train)**2)

        #计算梯度
        def dJ(theta):
            return np.dot(X_train.T, np.dot(X_train, theta) - y_train)

        X_train = np.hstack((X_train, np.ones((len(X_train),1))))
        last_theta = np.ones((X_train.shape[1]))
        learning_rate = learning_rate/np.mean(np.square(X_train), axis=0) # 统一量纲。也可以对数据先进行归一化
        cur_iter = 0
        while cur_iter < n_iters:
            gradient = dJ(last_theta) # 计算梯度
            theta = last_theta - learning_rate * gradient # 更新参数

            #每次训练时 的参数误差MSE
            y_train_predict=np.dot(X_train,theta)
            MSE_train = mean_squared_error(y_train,y_train_predict)
            print(f'第{cur_iter}次迭代的训练集MSE误差值为{MSE_train}')

            if (abs(J(theta) - J(last_theta)) < epsilon):  #目标函数更新小于一个足够小的值，则迭代结束
                break
            last_theta = theta
            cur_iter += 1
        self.theta = theta

    def predict(self, x_predict):
        x_predict = np.hstack((x_predict,np.ones((len(x_predict),1))))
        return np.dot(x_predict,self.theta)

reg = SimpleLinearRegression()
reg.fit(X_train, y_train, learning_rate=0.0005, n_iters=1e5, epsilon=1e-8)
print(reg.theta)

y_pred = reg.predict(X_test)
MSE_test = mean_squared_error(y_test,y_pred)
print("测试集MSE:",MSE_test)



第0次迭代的训练集MSE误差值为19846.30623538547
第1次迭代的训练集MSE误差值为14441.822874441326
第2次迭代的训练集MSE误差值为10879.293477332538
第3次迭代的训练集MSE误差值为8443.396642472748
第4次迭代的训练集MSE误差值为6763.775671965343
第5次迭代的训练集MSE误差值为5602.032593547922
第6次迭代的训练集MSE误差值为4796.805377873894
第7次迭代的训练集MSE误差值为4237.629795353587
第8次迭代的训练集MSE误差值为3848.5974182298933
第9次迭代的训练集MSE误差值为3577.426352831458
第10次迭代的训练集MSE误差值为3388.038991763332
第11次迭代的训练集MSE误差值为3255.496324656385
第12次迭代的训练集MSE误差值为3162.530703034866
第13次迭代的训练集MSE误差值为3097.1672306881396
第14次迭代的训练集MSE误差值为3051.088667739146
第15次迭代的训练集MSE误差值为3018.5093696598815
第16次迭代的训练集MSE误差值为2995.3984806873023
第17次迭代的训练集MSE误差值为2978.9432214722588
第18次迭代的训练集MSE误差值为2967.177521404704
第19次迭代的训练集MSE误差值为2958.7247026022483
第20次迭代的训练集MSE误差值为2952.6189520069215
第21次迭代的训练集MSE误差值为2948.18129668785
第22次迭代的训练集MSE误差值为2944.9333320166174
第23次迭代的训练集MSE误差值为2942.5371327605335
第24次迭代的训练集MSE误差值为2940.753344796801
第25次迭代的训练集MSE误差值为2939.411915848486
第26次迭代的训练集MSE误差值为2938.3916231816097
第27次迭代的训练集MSE误差值为2937.605731517662
第28次迭代的训练集MSE误差值为29